In [1]:
!pip install -q -U transformers datasets trl peft bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 68.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 24.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 10.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("b-mc2/sql-create-context",split="train[:25%]").train_test_split(test_size=0.06)
dataset

README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 18465
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 1179
    })
})

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

In [4]:
from huggingface_hub import login

login(token = secret_value_0)

In [5]:
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    quantization_config = bnb_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [8]:
tokenizer.pad_token = tokenizer.eos_token

In [13]:
import torch
p = '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 17 Feb 2025\n\nYou are a helpful assistant that helps generate SQL queries based on a table schema and a given question.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Which player played wide receiver after 2008?\nContext: CREATE TABLE table_name_17 (player VARCHAR, year VARCHAR, position VARCHAR)<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSQL Query: SELECT player FROM table_name_17 WHERE year > 2008 AND position = "wide receiver"<|eot_id|>'

tokens = tokenizer(p,return_tensors="pt").to("cuda")
response  = model.generate(**tokens)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [17]:
tokenizer.decode(response[0])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 17 Feb 2025\n\nYou are a helpful assistant that helps generate SQL queries based on a table schema and a given question.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Which player played wide receiver after 2008?\nContext: CREATE TABLE table_name_17 (player VARCHAR, year VARCHAR, position VARCHAR)<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSQL Query: SELECT player FROM table_name_17 WHERE year > 2008 AND position = "wide receiver"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThis SQL query will return the names of all players who played as wide receivers after'

In [18]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model,gradient_checkpointing_kwargs={"use_reentrant":False})

In [19]:
from peft import LoraConfig

rank_dimension = 8
lora_alpha = 16
lora_dropout = 0.05

peft_config = LoraConfig(
    r=rank_dimension,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    target_modules= ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

In [20]:
dataset['train'][0]

{'answer': 'SELECT date FROM table_name_14 WHERE game_site = "joe robbie stadium"',
 'question': 'What was the date of the game at Joe Robbie Stadium?',
 'context': 'CREATE TABLE table_name_14 (date VARCHAR, game_site VARCHAR)'}

In [21]:
def format_conversations(batch):

  messages = []

  for answer,question,context in zip(batch['answer'],batch['question'],batch['context']):

    message = [
        {"role" : "system", "content" : "You are a helpful assistant that helps generate SQL queries based on a table schema and a given question."},
        {"role" : "user", "content" : f"Question: {question}\nContext: {context}"},
        {"role" : "assistant", "content" : f"SQL Query: {answer}"}
    ]

    prompt = tokenizer.apply_chat_template(message,tokenize=False)
    messages.append(prompt)

  return {"text" : messages}

dataset = dataset.map(format_conversations,batched=True)
dataset

Map:   0%|          | 0/18465 [00:00<?, ? examples/s]

Map:   0%|          | 0/1179 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context', 'text'],
        num_rows: 18465
    })
    test: Dataset({
        features: ['answer', 'question', 'context', 'text'],
        num_rows: 1179
    })
})

In [22]:
dataset['train'][0]['text']

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 17 Feb 2025\n\nYou are a helpful assistant that helps generate SQL queries based on a table schema and a given question.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: What was the date of the game at Joe Robbie Stadium?\nContext: CREATE TABLE table_name_14 (date VARCHAR, game_site VARCHAR)<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSQL Query: SELECT date FROM table_name_14 WHERE game_site = "joe robbie stadium"<|eot_id|>'

In [23]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

# Training configuration
# Hyperparameters based on QLoRA paper recommendations
args = SFTConfig(
    output_dir="/output",
    max_steps = 400,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    learning_rate=2e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    logging_steps=25,
    eval_steps=25,
    eval_strategy = "steps",
    save_strategy="steps",
    bf16=True,
    push_to_hub=False,
    report_to="none",
    max_seq_length = 1024,
)

In [24]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    peft_config=peft_config,
    processing_class=tokenizer,
    args=args,
)

Applying chat template to train dataset:   0%|          | 0/18465 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/18465 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/18465 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/1179 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1179 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1179 [00:00<?, ? examples/s]

In [25]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
25,1.515900,0.923194
50,0.880900,0.792058
75,0.722200,0.672131
100,0.643800,0.659653
125,0.632500,0.648164
150,0.636300,0.633919
175,0.606000,0.622068
200,0.612800,0.618572
225,0.586400,0.612286
250,0.625200,0.606152


TrainOutput(global_step=400, training_loss=0.6822805690765381, metrics={'train_runtime': 5877.4438, 'train_samples_per_second': 0.272, 'train_steps_per_second': 0.068, 'total_flos': 3535448463544320.0, 'train_loss': 0.6822805690765381})

In [28]:
model.save_pretrained("lora_model")

## Push the model to HUB

In [33]:
from peft import AutoPeftModelForCausalLM


# Load PEFT model on CPU
model = AutoPeftModelForCausalLM.from_pretrained(
    model,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [143]:
merged_model_x = model.merge_and_unload()

In [151]:
merged_model_x.push_to_hub("sai-santhosh/text-2-sql-Llama-3.2-3B")

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/sai-santhosh/text-2-sql-Llama-3.2-3B/commit/65aab30d1c64a10db3e218f06ae9b05fc59c124e', commit_message='Upload LlamaForCausalLM', commit_description='', oid='65aab30d1c64a10db3e218f06ae9b05fc59c124e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sai-santhosh/text-2-sql-Llama-3.2-3B', endpoint='https://huggingface.co', repo_type='model', repo_id='sai-santhosh/text-2-sql-Llama-3.2-3B'), pr_revision=None, pr_num=None)

In [152]:
tokenizer.push_to_hub("sai-santhosh/text-2-sql-Llama-3.2-3B")

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sai-santhosh/text-2-sql-Llama-3.2-3B/commit/d6be88f0095906aeac8d354de5b32b464cb22cad', commit_message='Upload tokenizer', commit_description='', oid='d6be88f0095906aeac8d354de5b32b464cb22cad', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sai-santhosh/text-2-sql-Llama-3.2-3B', endpoint='https://huggingface.co', repo_type='model', repo_id='sai-santhosh/text-2-sql-Llama-3.2-3B'), pr_revision=None, pr_num=None)

## Inference

In [52]:
answer = "SELECT zip_code FROM weather GROUP BY zip_code HAVING AVG(mean_visibility_miles) < 10"
question ="Find the zip code in which the average mean visibility is lower than 10."
context = "CREATE TABLE weather (zip_code VARCHAR, mean_visibility_miles INTEGER)"

message = [
        {"role" : "system", "content" : "You are a helpful assistant that helps generate SQL queries based on a table schema and a given question."},
        {"role" : "user", "content" : f"Question: {question}\nContext: {context}"},
    ]

prompt = tokenizer.apply_chat_template(message,tokenize=False)
tokens = tokenizer(prompt,return_tensors="pt")
tokens

{'input_ids': tensor([[128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,  33025,
           2696,     25,   6790,    220,   2366,     18,    198,  15724,   2696,
             25,    220,   1114,  13806,    220,   2366,     20,    271,   2675,
            527,    264,  11190,  18328,    430,   8779,   7068,   8029,  20126,
           3196,    389,    264,   2007,  11036,    323,    264,   2728,   3488,
             13, 128009, 128006,    882, 128007,    271,  14924,     25,   7531,
            279,  10521,   2082,    304,    902,    279,   5578,   3152,  24035,
            374,   4827,   1109,    220,    605,    627,   2014,     25,  31876,
          14700,   9282,    320,  10169,   4229,  38689,     11,   3152,  72356,
            722,   3742,  31481,      8, 128009]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1

In [55]:
response = merged_model.generate(**tokens,max_length=500)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [56]:
tokenizer.decode(response[0])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 17 Feb 2025\n\nYou are a helpful assistant that helps generate SQL queries based on a table schema and a given question.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Find the zip code in which the average mean visibility is lower than 10.\nContext: CREATE TABLE weather (zip_code VARCHAR, mean_visibility_miles INTEGER)<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere is the SQL query to find the zip code in which the average mean visibility is lower than 10:\n\n```sql\nSELECT zip_code\nFROM weather\nWHERE mean_visibility_miles < 10;\n```\n\nThis query simply selects the `zip_code` column from the `weather` table where the `mean_visibility_miles` is less than 10. \n\nIf you want to get the average mean visibility for each zip code, you can use the following query:\n\n```sql\nSELECT zip_code, AVG(mean_visibility_miles) AS avg_v

In [62]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("new_token")

In [63]:
from huggingface_hub import login
login(token=token)

In [64]:
# push the merged model
merged_model.push_to_hub("sai-santhosh/text-2-sql-LLama-3.2")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sai-santhosh/text-2-sql-LLama-3.2/commit/5aff69eab5b406a95fa2860f7b7ada6db6cb6f07', commit_message='Upload LlamaForCausalLM', commit_description='', oid='5aff69eab5b406a95fa2860f7b7ada6db6cb6f07', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sai-santhosh/text-2-sql-LLama-3.2', endpoint='https://huggingface.co', repo_type='model', repo_id='sai-santhosh/text-2-sql-LLama-3.2'), pr_revision=None, pr_num=None)

In [65]:
# push the tokenizer
tokenizer.push_to_hub("sai-santhosh/text-2-sql-LLama-3.2")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sai-santhosh/text-2-sql-LLama-3.2/commit/be046d01a23f4cd77785354cc43348470527dd1d', commit_message='Upload tokenizer', commit_description='', oid='be046d01a23f4cd77785354cc43348470527dd1d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sai-santhosh/text-2-sql-LLama-3.2', endpoint='https://huggingface.co', repo_type='model', repo_id='sai-santhosh/text-2-sql-LLama-3.2'), pr_revision=None, pr_num=None)

In [66]:
# Push lora adapters 
model.push_to_hub("sai-santhosh/text-2-sql-LLama-3.2")

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sai-santhosh/text-2-sql-LLama-3.2/commit/9d8440f959d4697692adfc65bac351dd046bffa3', commit_message='Upload model', commit_description='', oid='9d8440f959d4697692adfc65bac351dd046bffa3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sai-santhosh/text-2-sql-LLama-3.2', endpoint='https://huggingface.co', repo_type='model', repo_id='sai-santhosh/text-2-sql-LLama-3.2'), pr_revision=None, pr_num=None)

## Inference  pipeline

In [69]:
from transformers import pipeline
pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=torch.cuda.current_device()
)

Device set to use cuda:0


In [74]:
def test_inference(question,context):
    prompt = pipe.tokenizer.apply_chat_template(
        [{"role": "user", "content": f"Question:\n {question} \n Context:\n {context}"}],
        tokenize=False,
        add_generation_prompt=True,
    )
    outputs = pipe(
        prompt,
        max_length = 500
    )
    return outputs

In [75]:
question ="Find the zip code in which the average mean visibility is lower than 10."
context = "CREATE TABLE weather (zip_code VARCHAR, mean_visibility_miles INTEGER)"

output = test_inference(question,context)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [112]:
from transformers import pipeline

pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=torch.cuda.current_device()
)

system_prompt = """Provide the SQL query to the question based on the context in below format
                                                    1. SQL Query: start with ```sql
                                                    2. Explanation of the query: start with ```explanation
                                                    
            make the explanation clear and detail in less than 75 words"""

def test_inference(question, context):
    prompt = pipe.tokenizer.apply_chat_template(
        [   {"role": "system", "content" : system_prompt},
            {"role": "user", "content": f"Question: {question} Context: {context}"}],
        tokenize=False,
        add_generation_prompt=True,
    )
    

    outputs = pipe(
        prompt,
        max_length=200, 
        clean_up_tokenization_spaces=True  
    )
    return outputs

question = "Find the zip code in which the average mean visibility is lower than 10."
context = "CREATE TABLE weather (zip_code VARCHAR, mean_visibility_miles INTEGER)"

output = test_inference(question, context)
print(output)

Device set to use cuda:0


[{'generated_text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 17 Feb 2025\n\nProvide the SQL query to the question based on the context in below format\n                                                    1. SQL Query: start with ```sql\n                                                    2. Explanation of the query: start with ```explanation\n                                                    \n            make the explanation clear and detail in less than 75 words<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Find the zip code in which the average mean visibility is lower than 10. Context: CREATE TABLE weather (zip_code VARCHAR, mean_visibility_miles INTEGER)<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n```sql\nSELECT zip_code\nFROM weather\nWHERE mean_visibility_miles < 10;\n```\n\n**Explanation:** This SQL query selects the `zip_code` from the `weather` table where the `mean_visi

### Combining the inference pipeline with preprocessing

In [129]:
from transformers import pipeline
import re

def get_sql_query(question,context):

    pipe = pipeline(
    "text-generation", model=merged_model, tokenizer=tokenizer, device=torch.cuda.current_device())

    system_prompt = """Provide the SQL query to the question based on the context in below format
                                                    1. SQL Query: start with ```sql
                                                    2. Explanation of the query: start with ```explanation
                                                    
            make the explanation clear and detail in less than 75 words"""

    prompt = pipe.tokenizer.apply_chat_template(
        [   {"role": "system", "content" : system_prompt},
            {"role": "user", "content": f"Question: {question} Context: {context}"}],
        tokenize=False,
        add_generation_prompt=True,
    )
    

    outputs = pipe(
        prompt,
        max_length=200, 
        clean_up_tokenization_spaces=True  
    )

    generated_text = outputs[0]['generated_text']

    generated_text = generated_text.replace(system_prompt,"")
    sql_query_pattern = r"```sql\n(.*?)\n```"
    sql_query_match = re.search(sql_query_pattern, generated_text, re.DOTALL)
    sql_query = sql_query_match.group(1).strip() if sql_query_match else None
    
    # Extract Explanation after **Explanation:**
    explanation_pattern = r"\*\*Explanation\:\*\*(.*)"
    explanation_match = re.search(explanation_pattern, generated_text, re.DOTALL)
    explanation = explanation_match.group(1).strip() if explanation_match else None
    
    print("Query:\n",sql_query)
    print("Explanation:\n",explanation)

In [131]:
get_sql_query(dataset['test'][10]['question'],dataset['test'][10]['context'])

Device set to use cuda:0


Query:
 SELECT club 
FROM table_name_48 
WHERE tries_for = '100';
Explanation:
 This SQL query selects the `club` column from the `table_name_48` table where the value in the `tries_for` column is '100'. This is a simple `SELECT` query that filters the data based on a specific condition. The `WHERE` clause
